In [35]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import os
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Embedding, Activation, LSTM, SimpleRNN, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from tensorflow.keras import backend as K
import tensorflow as tf
import tensorflow_hub as hub
print("TensorFlow Version:",tf.__version__)
print("Hub version: ",hub.__version__)
# Params for bert model and tokenization


TensorFlow Version: 2.1.0
Hub version:  0.8.0


In [36]:
class LoadingData():
            
    def __init__(self):
        train_file_path = os.path.join("..","input","nlp-benchmarking-data-for-intent-and-entity","benchmarking_data","Train")
        validation_file_path = os.path.join("..","input","nlp-benchmarking-data-for-intent-and-entity","benchmarking_data","Validate")
        category_id = 0
        self.cat_to_intent = {}
        self.intent_to_cat = {}
        
        for dirname, _, filenames in os.walk(train_file_path):
            for filename in filenames:
                file_path = os.path.join(dirname, filename)
                intent_id = filename.replace(".json","")
                self.cat_to_intent[category_id] = intent_id
                self.intent_to_cat[intent_id] = category_id
                category_id+=1
        print(self.cat_to_intent)
        print(self.intent_to_cat)
        '''Training data'''
        training_data = list() 
        for dirname, _, filenames in os.walk(train_file_path):
            for filename in filenames:
                file_path = os.path.join(dirname, filename)
                intent_id = filename.replace(".json","")
                training_data+=self.make_data_for_intent_from_json(file_path,intent_id,self.intent_to_cat[intent_id])
        self.train_data_frame = pd.DataFrame(training_data, columns =['query', 'intent','category'])   
        
        self.train_data_frame = self.train_data_frame.sample(frac = 1)


        
        '''Validation data'''
        validation_data = list()    
        for dirname, _, filenames in os.walk(validation_file_path):
            for filename in filenames:
                file_path = os.path.join(dirname, filename)
                intent_id = filename.replace(".json","")
                validation_data +=self.make_data_for_intent_from_json(file_path,intent_id,self.intent_to_cat[intent_id])                
        self.validation_data_frame = pd.DataFrame(validation_data, columns =['query', 'intent','category'])

        self.validation_data_frame = self.validation_data_frame.sample(frac = 1)
        
        
    def make_data_for_intent_from_json(self,json_file,intent_id,cat):
        json_d = json.load(open(json_file))         
        
        json_dict = json_d[intent_id]

        sent_list = list()
        for i in json_dict:
            each_list = i['data']
            sent =""
            for i in each_list:
                sent = sent + i['text']+ " "
            sent =sent[:-1]
            for i in range(3):
                sent = sent.replace("  "," ")
            sent_list.append((sent,intent_id,cat))
        return sent_list
            

In [37]:
load_data_obj = LoadingData()

{0: 'BookRestaurant', 1: 'SearchScreeningEvent', 2: 'RateBook', 3: 'GetWeather', 4: 'AddToPlaylist', 5: 'PlayMusic', 6: 'SearchCreativeWork'}
{'BookRestaurant': 0, 'SearchScreeningEvent': 1, 'RateBook': 2, 'GetWeather': 3, 'AddToPlaylist': 4, 'PlayMusic': 5, 'SearchCreativeWork': 6}


In [38]:
load_data_obj.train_data_frame.head()

,query,intent,category
13253,Find a painting called Beyond the Neighbourhood .,SearchCreativeWork,6
10462,Play Donald Rubinstein on Pandora,PlayMusic,5
6220,Will it be temperate in Tanana France in a week,GetWeather,3
8219,Put this Jerry Dixon song onto my Tokyo Rising...,AddToPlaylist,4
8561,Add this track to my Rock Hard playlist,AddToPlaylist,4


In [39]:
load_data_obj.train_data_frame

,query,intent,category
13253,Find a painting called Beyond the Neighbourhood .,SearchCreativeWork,6
10462,Play Donald Rubinstein on Pandora,PlayMusic,5
6220,Will it be temperate in Tanana France in a week,GetWeather,3
8219,Put this Jerry Dixon song onto my Tokyo Rising...,AddToPlaylist,4
8561,Add this track to my Rock Hard playlist,AddToPlaylist,4
...,...,...,...
711,Book me a reservation for 6 at a highly rated pub,BookRestaurant,0
823,I'd like to eat around ID on august the 16th w...,BookRestaurant,0
1707,Need a table at The Goof in Croatia for a part...,BookRestaurant,0
6082,"Tell me the weather forecast for Ethridge , Ma...",GetWeather,3


In [40]:
load_data_obj.validation_data_frame.head().values

array([['need a table somewhere in Quarryville 14 hours from now',
        'BookRestaurant', 0],
       ['Add a track to playlist Cena con Amigos', 'AddToPlaylist', 4],
       ["Please play a sound track from the fifties that's on Iheart",
        'PlayMusic', 5],
       ['what is the forecast in North Carolina for Edgemoor',
        'GetWeather', 3],
       ['add Beyond the Valley of 1984 in playlist Folk Music At The Gaslight Cafe',
        'AddToPlaylist', 4]], dtype=object)

In [41]:
load_data_obj.train_data_frame.head()

,query,intent,category
13253,Find a painting called Beyond the Neighbourhood .,SearchCreativeWork,6
10462,Play Donald Rubinstein on Pandora,PlayMusic,5
6220,Will it be temperate in Tanana France in a week,GetWeather,3
8219,Put this Jerry Dixon song onto my Tokyo Rising...,AddToPlaylist,4
8561,Add this track to my Rock Hard playlist,AddToPlaylist,4


# LSTM

In [42]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize the tokenizer
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(load_data_obj.train_data_frame['query'])

# Convert text to sequence of integers
train_sequences = tokenizer.texts_to_sequences(load_data_obj.train_data_frame['query'])
validation_sequences = tokenizer.texts_to_sequences(load_data_obj.validation_data_frame['query'])

# Pad sequences to ensure uniform length
max_length = max([len(x) for x in train_sequences])
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post')
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding='post')


In [43]:
import pandas as pd
from tensorflow.keras.utils import to_categorical

# Convert labels to one-hot encoding
train_labels = to_categorical(load_data_obj.train_data_frame['category'])
validation_labels = to_categorical(load_data_obj.validation_data_frame['category'])


In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

# Assuming max_length is defined here, e.g., max_length = max([len(x) for x in train_sequences])
max_length = max([len(x) for x in train_sequences])

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=32, input_length=max_length))  # Adjusted output_dim
model.add(Bidirectional(LSTM(128, return_sequences=False)))  # Increased LSTM units and added Bidirectional layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))  # Adjusted dropout rate
model.add(Dense(len(train_labels[0]), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 35, 32)            160000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               164864    
_________________________________________________________________
dense_7 (Dense)              (None, 64)                16448     
_________________________________________________________________
dropout_119 (Dropout)        (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 7)                 455       
Total params: 341,767
Trainable params: 341,767
Non-trainable params: 0
_________________________________________________________________


In [45]:
history = model.fit(train_padded, train_labels, epochs=10, validation_data=(validation_padded, validation_labels))


Train on 13784 samples, validate on 700 samples
Epoch 1/10
13784/13784 [==============================] - 6s 458us/sample - loss: 0.5106 - accuracy: 0.8220 - val_loss: 0.1009 - val_accuracy: 0.9686
Epoch 2/10
13784/13784 [==============================] - 4s 256us/sample - loss: 0.0783 - accuracy: 0.9772 - val_loss: 0.0855 - val_accuracy: 0.9729
Epoch 3/10
13784/13784 [==============================] - 3s 254us/sample - loss: 0.0537 - accuracy: 0.9841 - val_loss: 0.0816 - val_accuracy: 0.9771
Epoch 4/10
13784/13784 [==============================] - 4s 261us/sample - loss: 0.0331 - accuracy: 0.9910 - val_loss: 0.0737 - val_accuracy: 0.9700
Epoch 5/10
13784/13784 [==============================] - 4s 275us/sample - loss: 0.0241 - accuracy: 0.9932 - val_loss: 0.0719 - val_accuracy: 0.9771
Epoch 6/10
13784/13784 [==============================] - 4s 274us/sample - loss: 0.0161 - accuracy: 0.9954 - val_loss: 0.0780 - val_accuracy: 0.9729
Epoch 7/10
13784/13784 [============================

In [46]:
loss, accuracy = model.evaluate(validation_padded, validation_labels)
print(f'Validation loss: {loss}, Validation accuracy: {accuracy}')


700/700 [==============================] - 0s 110us/sample - loss: 0.0816 - accuracy: 0.9729
Validation loss: 0.08156434484890529, Validation accuracy: 0.9728571176528931


In [47]:

# Create a DataFrame for results
results_df = pd.DataFrame(columns=['model_name', 'validation_accuracy'])

# Insert data into the DataFrame
results_df = results_df.append({'model_name': 'LSTM', 'validation_accuracy': accuracy}, ignore_index=True)

# Display the DataFrame
print(results_df)


  model_name  validation_accuracy
0       LSTM             0.972857


# RANDOM FOREST


In [48]:
train_data_frame=load_data_obj.train_data_frame
validation_data_frame=load_data_obj.validation_data_frame

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
train_tfidf = tfidf_vectorizer.fit_transform(train_data_frame['query'])
validation_tfidf = tfidf_vectorizer.transform(validation_data_frame['query'])

# Encode labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_data_frame['category'])
validation_labels_encoded = label_encoder.transform(validation_data_frame['category'])


In [49]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_classifier.fit(train_tfidf, train_labels_encoded)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [50]:
# Predict on validation set
validation_predictions = rf_classifier.predict(validation_tfidf)

# Calculate accuracy
validation_accuracy = accuracy_score(validation_labels_encoded, validation_predictions)
print(f'Validation Accuracy of Random Forest: {validation_accuracy * 100:.2f}%')


Validation Accuracy of Random Forest: 97.86%


In [51]:
# Insert data into the DataFrame
results_df = results_df.append({'model_name': 'Random Forest', 'validation_accuracy': validation_accuracy}, ignore_index=True)

# Display the DataFrame
print(results_df)

      model_name  validation_accuracy
0           LSTM             0.972857
1  Random Forest             0.978571


# Logistic Regression

In [52]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Initialize the Logistic Regression Classifier
log_reg_classifier = LogisticRegression(max_iter=1000)  # Increase max_iter if the model doesn't converge

# Train the model
log_reg_classifier.fit(train_tfidf, train_labels_encoded)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [53]:
# Predict on validation set
validation_predictions = log_reg_classifier.predict(validation_tfidf)

# Calculate accuracy
validation_accuracy = accuracy_score(validation_labels_encoded, validation_predictions)
print(f'Validation Accuracy of Logistic Regression: {validation_accuracy * 100:.2f}%')


Validation Accuracy of Logistic Regression: 98.29%


In [54]:
# Insert data into the DataFrame
results_df = results_df.append({'model_name': 'Logistic Regression', 'validation_accuracy': validation_accuracy}, ignore_index=True)

# Display the DataFrame
print(results_df)


            model_name  validation_accuracy
0                 LSTM             0.972857
1        Random Forest             0.978571
2  Logistic Regression             0.982857


# GRU

In [55]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Initialize the model
model = Sequential()

# Add an Embedding layer
model.add(Embedding(input_dim=5000, output_dim=16, input_length=max_length))

# First GRU layer with Dropout regularization
model.add(GRU(units=50, return_sequences=True, activation='tanh'))
model.add(Dropout(0.2))

# Second GRU layer
model.add(GRU(units=50, return_sequences=True, activation='tanh'))
model.add(Dropout(0.2))

# Third GRU layer
model.add(GRU(units=50, return_sequences=True, activation='tanh'))
model.add(Dropout(0.2))

# Fourth GRU layer
model.add(GRU(units=50, activation='tanh'))
model.add(Dropout(0.2))

# Output layer for classification (units = number of classes, softmax activation)
model.add(Dense(units=len(train_labels[0]), activation='softmax'))  # Adjust the units based on the number of classes

# Compile the model for classification
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 35, 16)            80000     
_________________________________________________________________
gru_4 (GRU)                  (None, 35, 50)            10200     
_________________________________________________________________
dropout_120 (Dropout)        (None, 35, 50)            0         
_________________________________________________________________
gru_5 (GRU)                  (None, 35, 50)            15300     
_________________________________________________________________
dropout_121 (Dropout)        (None, 35, 50)            0         
_________________________________________________________________
gru_6 (GRU)                  (None, 35, 50)            15300     
_________________________________________________________________
dropout_122 (Dropout)        (None, 35, 50)           

In [56]:
# Train the model
history = model.fit(train_padded, train_labels, epochs=10, validation_data=(validation_padded, validation_labels))


Train on 13784 samples, validate on 700 samples
Epoch 1/10
13784/13784 [==============================] - 11s 784us/sample - loss: 1.9474 - accuracy: 0.1411 - val_loss: 1.9464 - val_accuracy: 0.1429
Epoch 2/10
13784/13784 [==============================] - 6s 408us/sample - loss: 1.9473 - accuracy: 0.1414 - val_loss: 1.9463 - val_accuracy: 0.1429
Epoch 3/10
13784/13784 [==============================] - 6s 413us/sample - loss: 0.8739 - accuracy: 0.6518 - val_loss: 0.2071 - val_accuracy: 0.9471
Epoch 4/10
13784/13784 [==============================] - 6s 410us/sample - loss: 0.1849 - accuracy: 0.9529 - val_loss: 0.1493 - val_accuracy: 0.9543
Epoch 5/10
13784/13784 [==============================] - 6s 413us/sample - loss: 0.1188 - accuracy: 0.9706 - val_loss: 0.1310 - val_accuracy: 0.9671
Epoch 6/10
13784/13784 [==============================] - 6s 408us/sample - loss: 0.0931 - accuracy: 0.9783 - val_loss: 0.1180 - val_accuracy: 0.9743
Epoch 7/10
13784/13784 [===========================

In [57]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_padded, validation_labels)
print(f'Validation loss: {loss}, Validation accuracy: {accuracy}')


700/700 [==============================] - 0s 154us/sample - loss: 0.1968 - accuracy: 0.9500
Validation loss: 0.1967534668317863, Validation accuracy: 0.949999988079071


In [58]:
# Insert data into the DataFrame
results_df = results_df.append({'model_name': 'GRU', 'validation_accuracy': accuracy}, ignore_index=True)

# Display the DataFrame
print(results_df)


            model_name  validation_accuracy
0                 LSTM             0.972857
1        Random Forest             0.978571
2  Logistic Regression             0.982857
3                  GRU             0.950000


# RNN 

In [59]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Initialize the model
model = Sequential()

# Add an Embedding layer
model.add(Embedding(input_dim=5000, output_dim=16, input_length=max_length))

# Add a SimpleRNN layer
model.add(SimpleRNN(units=64, return_sequences=True))
model.add(Dropout(0.2))

# Add another SimpleRNN layer
model.add(SimpleRNN(units=64))
model.add(Dropout(0.2))

# Add the output Dense layer with softmax activation for multi-class classification
model.add(Dense(units=len(train_labels[0]), activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 35, 16)            80000     
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 35, 64)            5184      
_________________________________________________________________
dropout_124 (Dropout)        (None, 35, 64)            0         
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 64)                8256      
_________________________________________________________________
dropout_125 (Dropout)        (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 7)                 455       
Total params: 93,895
Trainable params: 93,895
Non-trainable params: 0
__________________________________________________

In [60]:
# Train the model
history = model.fit(train_padded, train_labels, epochs=10, validation_data=(validation_padded, validation_labels))


Train on 13784 samples, validate on 700 samples
Epoch 1/10
13784/13784 [==============================] - 17s 1ms/sample - loss: 0.5194 - accuracy: 0.8371 - val_loss: 0.2853 - val_accuracy: 0.9214
Epoch 2/10
13784/13784 [==============================] - 15s 1ms/sample - loss: 0.1414 - accuracy: 0.9594 - val_loss: 0.1566 - val_accuracy: 0.9543
Epoch 3/10
13784/13784 [==============================] - 14s 1ms/sample - loss: 0.0779 - accuracy: 0.9774 - val_loss: 0.1649 - val_accuracy: 0.9600
Epoch 4/10
13784/13784 [==============================] - 15s 1ms/sample - loss: 0.0594 - accuracy: 0.9834 - val_loss: 0.1892 - val_accuracy: 0.9557
Epoch 5/10
13784/13784 [==============================] - 14s 1ms/sample - loss: 0.0582 - accuracy: 0.9828 - val_loss: 0.2602 - val_accuracy: 0.9371
Epoch 6/10
13784/13784 [==============================] - 14s 1ms/sample - loss: 0.0464 - accuracy: 0.9861 - val_loss: 0.1732 - val_accuracy: 0.9557
Epoch 7/10
13784/13784 [==============================] - 

In [61]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_padded, validation_labels)
print(f'Validation loss: {loss}, Validation accuracy: {accuracy}')


700/700 [==============================] - 0s 207us/sample - loss: 0.1926 - accuracy: 0.9571
Validation loss: 0.19259030122842108, Validation accuracy: 0.9571428298950195


In [62]:
# Insert data into the DataFrame
results_df = results_df.append({'model_name': 'RNN', 'validation_accuracy': accuracy}, ignore_index=True)

# Display the DataFrame
print(results_df)


            model_name  validation_accuracy
0                 LSTM             0.972857
1        Random Forest             0.978571
2  Logistic Regression             0.982857
3                  GRU             0.950000
4                  RNN             0.957143


# BERT

In [64]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer


# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

# Load and preprocess the data
data_train = train_data_frame[['query', 'intent']]
data_train['category'] = pd.Categorical(data_train['intent'])
data_train['intent'] = data_train['category'].cat.codes

# Load and preprocess the data
data_test = validation_data_frame[['query', 'intent']]
data_test['category'] = pd.Categorical(data_test['intent'])
data_test['intent'] = data_test['category'].cat.codes



# Extract the training and testing texts and labels
train_texts = data_train['query'].tolist()
train_labels = data_train['intent'].tolist()
test_texts = data_test['query'].tolist()
test_labels = data_test['intent'].tolist()

max_length = 128  # Adjust based on your dataset or model's max length

train_encodings = tokenizer.batch_encode_plus(
    train_texts, 
    truncation=True, 
    padding='max_length', 
    max_length=max_length, 
    return_tensors="tf",
    pad_to_max_length=True  # Explicitly enforce padding to max_length
)

test_encodings = tokenizer.batch_encode_plus(
    test_texts, 
    truncation=True, 
    padding='max_length', 
    max_length=max_length, 
    return_tensors="tf",
    pad_to_max_length=True  # Explicitly enforce padding to max_length
)



# Convert the labels to one-hot encoding
num_labels = len(data_train['category'].cat.categories)
train_labels_encoded = tf.one_hot(train_labels, num_labels)
test_labels_encoded = tf.one_hot(test_labels, num_labels)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels_encoded))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels_encoded))

# Define the model architecture
input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='attention_mask')
output = model(input_ids, attention_mask=attention_mask)[0]
output = tf.keras.layers.Dense(num_labels, activation='softmax')(output[:, 0, :])  # Pooling the output
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

# Compile and train the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']



# Use smaller batch size
batch_size = 20



model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model.fit(train_dataset.batch(batch_size), epochs=10)

# Evaluate the model
model.evaluate(test_dataset.batch(batch_size))

Train for 690 steps
Epoch 1/10
690/690 [==============================] - 244s 354ms/step - loss: 1.2345 - accuracy: 0.9401
Epoch 2/10
690/690 [==============================] - 228s 330ms/step - loss: 1.1788 - accuracy: 0.9872
Epoch 3/10
690/690 [==============================] - 228s 331ms/step - loss: 1.1762 - accuracy: 0.9895
Epoch 4/10
690/690 [==============================] - 228s 330ms/step - loss: 1.1733 - accuracy: 0.9920
Epoch 5/10
690/690 [==============================] - 228s 330ms/step - loss: 1.1734 - accuracy: 0.9920
Epoch 6/10
690/690 [==============================] - 228s 330ms/step - loss: 1.1725 - accuracy: 0.9928
Epoch 7/10
690/690 [==============================] - 228s 330ms/step - loss: 1.1720 - accuracy: 0.9936
Epoch 8/10
690/690 [==============================] - 228s 330ms/step - loss: 1.1717 - accuracy: 0.9938
Epoch 9/10
690/690 [==============================] - 228s 331ms/step - loss: 1.1722 - accuracy: 0.9932
Epoch 10/10
35/35 [=========================

[1.1755036558423724, 0.99]

In [65]:
evaluation_results = model.evaluate(test_dataset.batch(batch_size))
accuracy = evaluation_results[1]  # Assuming accuracy is the second metric in the metrics list


35/35 [==============================] - 4s 124ms/step - loss: 1.1755 - accuracy: 0.9900


In [66]:
# Insert data into the DataFrame
results_df = results_df.append({'model_name': 'BERT', 'validation_accuracy': accuracy}, ignore_index=True)

# Display the DataFrame
print(results_df)


            model_name  validation_accuracy
0                 LSTM             0.972857
1        Random Forest             0.978571
2  Logistic Regression             0.982857
3                  GRU             0.950000
4                  RNN             0.957143
5                 BERT             0.990000


# ROBERTA


In [67]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import TFRobertaModel, RobertaTokenizer

#Load the tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = TFRobertaModel.from_pretrained('roberta-base')



# Load and preprocess the data
data_train = train_data_frame[['query', 'intent']]
data_train['category'] = pd.Categorical(data_train['intent'])
data_train['intent'] = data_train['category'].cat.codes

# Load and preprocess the data
data_test = validation_data_frame[['query', 'intent']]
data_test['category'] = pd.Categorical(data_test['intent'])
data_test['intent'] = data_test['category'].cat.codes



# Extract the training and testing texts and labels
train_texts = data_train['query'].tolist()
train_labels = data_train['intent'].tolist()
test_texts = data_test['query'].tolist()
test_labels = data_test['intent'].tolist()

max_length = 128  # Adjust based on your dataset or model's max length

train_encodings = tokenizer.batch_encode_plus(
    train_texts, 
    truncation=True, 
    padding='max_length', 
    max_length=max_length, 
    return_tensors="tf",
    pad_to_max_length=True  # Explicitly enforce padding to max_length
)

test_encodings = tokenizer.batch_encode_plus(
    test_texts, 
    truncation=True, 
    padding='max_length', 
    max_length=max_length, 
    return_tensors="tf",
    pad_to_max_length=True  # Explicitly enforce padding to max_length
)



# Convert the labels to one-hot encoding
num_labels = len(data_train['category'].cat.categories)
train_labels_encoded = tf.one_hot(train_labels, num_labels)
test_labels_encoded = tf.one_hot(test_labels, num_labels)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels_encoded))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels_encoded))

#Define the model architecture
input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='attention_mask')
output = model(input_ids, attention_mask=attention_mask)[0]
output = tf.keras.layers.Dense(num_labels, activation='softmax')(output[:, 0, :]) # Pooling the output
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

#Compile and train the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']


# Use smaller batch size
batch_size = 20



model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model.fit(train_dataset.batch(batch_size), epochs=10)

# Evaluate the model

evaluation_results = model.evaluate(test_dataset.batch(batch_size))
accuracy = evaluation_results[1]  # Assuming accuracy is the second metric in the metrics list

Train for 690 steps
Epoch 1/10
690/690 [==============================] - 247s 358ms/step - loss: 1.2303 - accuracy: 0.9393
Epoch 2/10
690/690 [==============================] - 231s 335ms/step - loss: 1.1798 - accuracy: 0.9860
Epoch 3/10
690/690 [==============================] - 231s 334ms/step - loss: 1.1774 - accuracy: 0.9882
Epoch 4/10
690/690 [==============================] - 231s 334ms/step - loss: 1.1769 - accuracy: 0.9885
Epoch 5/10
690/690 [==============================] - 231s 334ms/step - loss: 1.1752 - accuracy: 0.9904
Epoch 6/10
690/690 [==============================] - 231s 334ms/step - loss: 1.1743 - accuracy: 0.9914
Epoch 7/10
690/690 [==============================] - 231s 334ms/step - loss: 1.1753 - accuracy: 0.9901
Epoch 8/10
690/690 [==============================] - 231s 335ms/step - loss: 1.1755 - accuracy: 0.9900
Epoch 9/10
690/690 [==============================] - 231s 335ms/step - loss: 1.1758 - accuracy: 0.9898
Epoch 10/10
35/35 [=========================

In [68]:
# Insert data into the DataFrame
results_df = results_df.append({'model_name': 'Roberta', 'validation_accuracy': accuracy}, ignore_index=True)

# Display the DataFrame
print(results_df)


            model_name  validation_accuracy
0                 LSTM             0.972857
1        Random Forest             0.978571
2  Logistic Regression             0.982857
3                  GRU             0.950000
4                  RNN             0.957143
5                 BERT             0.990000
6              Roberta             0.991429


# XLnet 

In [69]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import TFXLNetModel, XLNetTokenizer

# Load the tokenizer and model
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = TFXLNetModel.from_pretrained('xlnet-base-cased')




# Load and preprocess the data
data_train = train_data_frame[['query', 'intent']]
data_train['category'] = pd.Categorical(data_train['intent'])
data_train['intent'] = data_train['category'].cat.codes

# Load and preprocess the data
data_test = validation_data_frame[['query', 'intent']]
data_test['category'] = pd.Categorical(data_test['intent'])
data_test['intent'] = data_test['category'].cat.codes



# Extract the training and testing texts and labels
train_texts = data_train['query'].tolist()
train_labels = data_train['intent'].tolist()
test_texts = data_test['query'].tolist()
test_labels = data_test['intent'].tolist()

max_length = 128  # Adjust based on your dataset or model's max length

train_encodings = tokenizer.batch_encode_plus(
    train_texts, 
    truncation=True, 
    padding='max_length', 
    max_length=max_length, 
    return_tensors="tf",
    pad_to_max_length=True  # Explicitly enforce padding to max_length
)

test_encodings = tokenizer.batch_encode_plus(
    test_texts, 
    truncation=True, 
    padding='max_length', 
    max_length=max_length, 
    return_tensors="tf",
    pad_to_max_length=True  # Explicitly enforce padding to max_length
)



# Convert the labels to one-hot encoding
num_labels = len(data_train['category'].cat.categories)
train_labels_encoded = tf.one_hot(train_labels, num_labels)
test_labels_encoded = tf.one_hot(test_labels, num_labels)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels_encoded))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels_encoded))

#Define the model architecture
input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='attention_mask')
output = model(input_ids, attention_mask=attention_mask)[0]
output = tf.keras.layers.Dense(num_labels, activation='softmax')(output[:, 0, :]) # Pooling the output
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

#Compile and train the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']


# Use smaller batch size
batch_size = 20



model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model.fit(train_dataset.batch(batch_size), epochs=10)

# Evaluate the model

evaluation_results = model.evaluate(test_dataset.batch(batch_size))
accuracy = evaluation_results[1]  # Assuming accuracy is the second metric in the metrics list

Train for 690 steps
Epoch 1/10
690/690 [==============================] - 305s 442ms/step - loss: 1.3841 - accuracy: 0.7779
Epoch 2/10
690/690 [==============================] - 292s 423ms/step - loss: 1.1936 - accuracy: 0.9718
Epoch 3/10
690/690 [==============================] - 292s 423ms/step - loss: 1.1864 - accuracy: 0.9787
Epoch 4/10
690/690 [==============================] - 292s 423ms/step - loss: 1.1855 - accuracy: 0.9801
Epoch 5/10
690/690 [==============================] - 292s 423ms/step - loss: 1.1819 - accuracy: 0.9835
Epoch 6/10
690/690 [==============================] - 292s 423ms/step - loss: 1.1804 - accuracy: 0.9851
Epoch 7/10
690/690 [==============================] - 292s 423ms/step - loss: 1.1792 - accuracy: 0.9861
Epoch 8/10
690/690 [==============================] - 292s 423ms/step - loss: 1.1803 - accuracy: 0.9852
Epoch 9/10
690/690 [==============================] - 292s 423ms/step - loss: 1.1784 - accuracy: 0.9871
Epoch 10/10
35/35 [=========================

In [70]:
# Insert data into the DataFrame
results_df = results_df.append({'model_name': 'XLnet', 'validation_accuracy': accuracy}, ignore_index=True)

# Display the DataFrame
print(results_df)


            model_name  validation_accuracy
0                 LSTM             0.972857
1        Random Forest             0.978571
2  Logistic Regression             0.982857
3                  GRU             0.950000
4                  RNN             0.957143
5                 BERT             0.990000
6              Roberta             0.991429
7                XLnet             0.994286


# DistilBert


In [71]:
from transformers import TFDistilBertModel, DistilBertTokenizer

#Load the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')




# Load and preprocess the data
data_train = train_data_frame[['query', 'intent']]
data_train['category'] = pd.Categorical(data_train['intent'])
data_train['intent'] = data_train['category'].cat.codes

# Load and preprocess the data
data_test = validation_data_frame[['query', 'intent']]
data_test['category'] = pd.Categorical(data_test['intent'])
data_test['intent'] = data_test['category'].cat.codes



# Extract the training and testing texts and labels
train_texts = data_train['query'].tolist()
train_labels = data_train['intent'].tolist()
test_texts = data_test['query'].tolist()
test_labels = data_test['intent'].tolist()

max_length = 128  # Adjust based on your dataset or model's max length

train_encodings = tokenizer.batch_encode_plus(
    train_texts, 
    truncation=True, 
    padding='max_length', 
    max_length=max_length, 
    return_tensors="tf",
    pad_to_max_length=True  # Explicitly enforce padding to max_length
)

test_encodings = tokenizer.batch_encode_plus(
    test_texts, 
    truncation=True, 
    padding='max_length', 
    max_length=max_length, 
    return_tensors="tf",
    pad_to_max_length=True  # Explicitly enforce padding to max_length
)



# Convert the labels to one-hot encoding
num_labels = len(data_train['category'].cat.categories)
train_labels_encoded = tf.one_hot(train_labels, num_labels)
test_labels_encoded = tf.one_hot(test_labels, num_labels)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels_encoded))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels_encoded))

#Define the model architecture
input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='attention_mask')
output = model(input_ids, attention_mask=attention_mask)[0]
output = tf.keras.layers.Dense(num_labels, activation='softmax')(output[:, 0, :]) # Pooling the output
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

#Compile and train the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']


# Use smaller batch size
batch_size = 20



model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model.fit(train_dataset.batch(batch_size), epochs=10)

# Evaluate the model

evaluation_results = model.evaluate(test_dataset.batch(batch_size))
accuracy = evaluation_results[1]  # Assuming accuracy is the second metric in the metrics list


Train for 690 steps
Epoch 1/10
690/690 [==============================] - 124s 179ms/step - loss: 1.2423 - accuracy: 0.9335
Epoch 2/10
690/690 [==============================] - 116s 169ms/step - loss: 1.1817 - accuracy: 0.9842
Epoch 3/10
690/690 [==============================] - 117s 169ms/step - loss: 1.1757 - accuracy: 0.9901
Epoch 4/10
690/690 [==============================] - 117s 169ms/step - loss: 1.1743 - accuracy: 0.9912
Epoch 5/10
690/690 [==============================] - 117s 169ms/step - loss: 1.1736 - accuracy: 0.9922
Epoch 6/10
690/690 [==============================] - 116s 169ms/step - loss: 1.1728 - accuracy: 0.9928
Epoch 7/10
690/690 [==============================] - 117s 169ms/step - loss: 1.1720 - accuracy: 0.9933
Epoch 8/10
690/690 [==============================] - 116s 169ms/step - loss: 1.1719 - accuracy: 0.9935
Epoch 9/10
690/690 [==============================] - 116s 169ms/step - loss: 1.1720 - accuracy: 0.9934
Epoch 10/10
35/35 [========================

In [72]:
# Insert data into the DataFrame
results_df = results_df.append({'model_name': 'DistilBert', 'validation_accuracy': accuracy}, ignore_index=True)

# Display the DataFrame
print(results_df)


            model_name  validation_accuracy
0                 LSTM             0.972857
1        Random Forest             0.978571
2  Logistic Regression             0.982857
3                  GRU             0.950000
4                  RNN             0.957143
5                 BERT             0.990000
6              Roberta             0.991429
7                XLnet             0.994286
8           DistilBert             0.988571


In [73]:
print(results_df)


            model_name  validation_accuracy
0                 LSTM             0.972857
1        Random Forest             0.978571
2  Logistic Regression             0.982857
3                  GRU             0.950000
4                  RNN             0.957143
5                 BERT             0.990000
6              Roberta             0.991429
7                XLnet             0.994286
8           DistilBert             0.988571


# Albert

In [74]:
from transformers import TFAlbertModel, AlbertTokenizer

tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = TFAlbertModel.from_pretrained('albert-base-v2')


# Load and preprocess the data
data_train = train_data_frame[['query', 'intent']]
data_train['category'] = pd.Categorical(data_train['intent'])
data_train['intent'] = data_train['category'].cat.codes

# Load and preprocess the data
data_test = validation_data_frame[['query', 'intent']]
data_test['category'] = pd.Categorical(data_test['intent'])
data_test['intent'] = data_test['category'].cat.codes



# Extract the training and testing texts and labels
train_texts = data_train['query'].tolist()
train_labels = data_train['intent'].tolist()
test_texts = data_test['query'].tolist()
test_labels = data_test['intent'].tolist()

max_length = 128  # Adjust based on your dataset or model's max length

train_encodings = tokenizer.batch_encode_plus(
    train_texts, 
    truncation=True, 
    padding='max_length', 
    max_length=max_length, 
    return_tensors="tf",
    pad_to_max_length=True  # Explicitly enforce padding to max_length
)

test_encodings = tokenizer.batch_encode_plus(
    test_texts, 
    truncation=True, 
    padding='max_length', 
    max_length=max_length, 
    return_tensors="tf",
    pad_to_max_length=True  # Explicitly enforce padding to max_length
)



# Convert the labels to one-hot encoding
num_labels = len(data_train['category'].cat.categories)
train_labels_encoded = tf.one_hot(train_labels, num_labels)
test_labels_encoded = tf.one_hot(test_labels, num_labels)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels_encoded))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels_encoded))

#Define the model architecture
input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='attention_mask')
output = model(input_ids, attention_mask=attention_mask)[0]
output = tf.keras.layers.Dense(num_labels, activation='softmax')(output[:, 0, :]) # Pooling the output
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

#Compile and train the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']


# Use smaller batch size
batch_size = 20



model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model.fit(train_dataset.batch(batch_size), epochs=10)

# Evaluate the model

evaluation_results = model.evaluate(test_dataset.batch(batch_size))
accuracy = evaluation_results[1]  # Assuming accuracy is the second metric in the metrics list


Train for 690 steps
Epoch 1/10
690/690 [==============================] - 231s 334ms/step - loss: 1.2816 - accuracy: 0.8855
Epoch 2/10
690/690 [==============================] - 219s 317ms/step - loss: 1.1920 - accuracy: 0.9738
Epoch 3/10
690/690 [==============================] - 219s 317ms/step - loss: 1.1867 - accuracy: 0.9791
Epoch 4/10
690/690 [==============================] - 219s 317ms/step - loss: 1.1869 - accuracy: 0.9787
Epoch 5/10
690/690 [==============================] - 219s 317ms/step - loss: 1.1849 - accuracy: 0.9806
Epoch 6/10
690/690 [==============================] - 219s 317ms/step - loss: 1.1846 - accuracy: 0.9807
Epoch 7/10
690/690 [==============================] - 219s 317ms/step - loss: 1.1801 - accuracy: 0.9853
Epoch 8/10
690/690 [==============================] - 219s 317ms/step - loss: 1.1788 - accuracy: 0.9867
Epoch 9/10
690/690 [==============================] - 219s 317ms/step - loss: 1.1800 - accuracy: 0.9854
Epoch 10/10
35/35 [========================

In [75]:
# Insert data into the DataFrame
results_df = results_df.append({'model_name': 'Albert', 'validation_accuracy': accuracy}, ignore_index=True)

# Display the DataFrame
print(results_df)


            model_name  validation_accuracy
0                 LSTM             0.972857
1        Random Forest             0.978571
2  Logistic Regression             0.982857
3                  GRU             0.950000
4                  RNN             0.957143
5                 BERT             0.990000
6              Roberta             0.991429
7                XLnet             0.994286
8           DistilBert             0.988571
9               Albert             0.987143


# Electra

In [76]:
from transformers import TFElectraModel, ElectraTokenizer

tokenizer = ElectraTokenizer.from_pretrained('google/electra-base-discriminator')
model = TFElectraModel.from_pretrained('google/electra-base-discriminator')



# Load and preprocess the data
data_train = train_data_frame[['query', 'intent']]
data_train['category'] = pd.Categorical(data_train['intent'])
data_train['intent'] = data_train['category'].cat.codes

# Load and preprocess the data
data_test = validation_data_frame[['query', 'intent']]
data_test['category'] = pd.Categorical(data_test['intent'])
data_test['intent'] = data_test['category'].cat.codes



# Extract the training and testing texts and labels
train_texts = data_train['query'].tolist()
train_labels = data_train['intent'].tolist()
test_texts = data_test['query'].tolist()
test_labels = data_test['intent'].tolist()

max_length = 128  # Adjust based on your dataset or model's max length

train_encodings = tokenizer.batch_encode_plus(
    train_texts, 
    truncation=True, 
    padding='max_length', 
    max_length=max_length, 
    return_tensors="tf",
    pad_to_max_length=True  # Explicitly enforce padding to max_length
)

test_encodings = tokenizer.batch_encode_plus(
    test_texts, 
    truncation=True, 
    padding='max_length', 
    max_length=max_length, 
    return_tensors="tf",
    pad_to_max_length=True  # Explicitly enforce padding to max_length
)



# Convert the labels to one-hot encoding
num_labels = len(data_train['category'].cat.categories)
train_labels_encoded = tf.one_hot(train_labels, num_labels)
test_labels_encoded = tf.one_hot(test_labels, num_labels)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels_encoded))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels_encoded))

#Define the model architecture
input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='attention_mask')
output = model(input_ids, attention_mask=attention_mask)[0]
output = tf.keras.layers.Dense(num_labels, activation='softmax')(output[:, 0, :]) # Pooling the output
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

#Compile and train the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']


# Use smaller batch size
batch_size = 20



model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model.fit(train_dataset.batch(batch_size), epochs=10)

# Evaluate the model

evaluation_results = model.evaluate(test_dataset.batch(batch_size))
accuracy = evaluation_results[1]  # Assuming accuracy is the second metric in the metrics list


Train for 690 steps
Epoch 1/10
690/690 [==============================] - 243s 353ms/step - loss: 1.2771 - accuracy: 0.9069
Epoch 2/10
690/690 [==============================] - 228s 330ms/step - loss: 1.1831 - accuracy: 0.9836
Epoch 3/10
690/690 [==============================] - 228s 330ms/step - loss: 1.1796 - accuracy: 0.9860
Epoch 4/10
690/690 [==============================] - 228s 330ms/step - loss: 1.1765 - accuracy: 0.9893
Epoch 5/10
690/690 [==============================] - 228s 330ms/step - loss: 1.1779 - accuracy: 0.9877
Epoch 6/10
690/690 [==============================] - 228s 330ms/step - loss: 1.1760 - accuracy: 0.9896
Epoch 7/10
690/690 [==============================] - 228s 330ms/step - loss: 1.1750 - accuracy: 0.9905
Epoch 8/10
690/690 [==============================] - 228s 330ms/step - loss: 1.1744 - accuracy: 0.9910
Epoch 9/10
690/690 [==============================] - 228s 330ms/step - loss: 1.1747 - accuracy: 0.9907
Epoch 10/10
35/35 [========================

In [77]:
# Insert data into the DataFrame
results_df = results_df.append({'model_name': 'Electra', 'validation_accuracy': accuracy}, ignore_index=True)

# Display the DataFrame
print(results_df)


             model_name  validation_accuracy
0                  LSTM             0.972857
1         Random Forest             0.978571
2   Logistic Regression             0.982857
3                   GRU             0.950000
4                   RNN             0.957143
5                  BERT             0.990000
6               Roberta             0.991429
7                 XLnet             0.994286
8            DistilBert             0.988571
9                Albert             0.987143
10              Electra             0.985714


# Final Evaluation

In [78]:
# Find the index of the row with the maximum validation accuracy
max_accuracy_index = results_df['validation_accuracy'].idxmax()

# Retrieve the model name with the maximum validation accuracy
best_model_name = results_df.loc[max_accuracy_index, 'model_name']

print(f"The best model with maximum validation accuracy is: {best_model_name}")

The best model with maximum validation accuracy is: XLnet
